In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [4]:
df = pd.read_csv("sales_pos.csv")
df.head()

,user,prod,gender,age_group,job,city,marital,prod_cat1,prod_cat2,prod_cat3,purchase
0,1,P00069042,F,0-17,10,A,0,3,NaN,NaN,8370
1,1,P00248942,F,0-17,10,A,0,1,6.0,14.0,15200
2,1,P00087842,F,0-17,10,A,0,12,NaN,NaN,1422
3,1,P00085442,F,0-17,10,A,0,12,14.0,NaN,1057
4,2,P00285442,M,55+,16,C,0,8,NaN,NaN,7969


# Q1

In [7]:
df1 = df.copy()
df1.groupby("prod")["purchase"].sum().max()

27995166

df1.groupby("prod")["purchase"].sum().sort_values()

In [100]:
df1.groupby("prod")["purchase"].sum().idxmax()

'P00025442'

In [11]:
df11 = df1.groupby("prod")["purchase"].sum().reset_index()
df11.loc[df11["purchase"] == df11["purchase"].max(),:]

,prod,purchase
249,P00025442,27995166


In [21]:
df12 = df1.loc[df1["prod"] == "P00025442",:]
df12

,user,prod,gender,age_group,job,city,marital,prod_cat1,prod_cat2,prod_cat3,purchase
667,130,P00025442,M,36-45,17,B,1,1,2.0,9.0,19706
749,142,P00025442,M,26-35,7,A,0,1,2.0,9.0,15212
833,150,P00025442,M,36-45,7,B,1,1,2.0,9.0,15255
1134,192,P00025442,M,18-25,1,B,0,1,2.0,9.0,15223
1205,198,P00025442,M,26-35,12,A,1,1,2.0,9.0,19296
...,...,...,...,...,...,...,...,...,...,...,...
544755,5854,P00025442,M,46-50,7,B,1,1,2.0,9.0,19072
544780,5858,P00025442,M,26-35,4,B,0,1,2.0,9.0,19665
545079,5911,P00025442,F,26-35,17,C,1,1,2.0,9.0,15284
545206,5933,P00025442,M,26-35,2,C,1,1,2.0,9.0,15338


In [26]:
df12["cnt"] = 1
df13 = df12.groupby("job")["cnt"].count().reset_index().sort_values("cnt")
df13

C:\Users\younl\AppData\Local\Temp/ipykernel_14316/4064115313.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df12["cnt"] = 1


,job,cnt
8,8,3
9,9,12
18,18,14
19,19,20
13,13,24
5,5,31
11,11,38
3,3,46
15,15,50
6,6,54


# Q2

In [33]:
df2 = df.copy()
df2

,user,prod,gender,age_group,job,city,marital,prod_cat1,prod_cat2,prod_cat3,purchase
0,1,P00069042,F,0-17,10,A,0,3,NaN,NaN,8370
1,1,P00248942,F,0-17,10,A,0,1,6.0,14.0,15200
2,1,P00087842,F,0-17,10,A,0,12,NaN,NaN,1422
3,1,P00085442,F,0-17,10,A,0,12,14.0,NaN,1057
4,2,P00285442,M,55+,16,C,0,8,NaN,NaN,7969
...,...,...,...,...,...,...,...,...,...,...,...
550063,6033,P00372445,M,51-55,13,B,1,20,NaN,NaN,368
550064,6035,P00375436,F,26-35,1,C,0,20,NaN,NaN,371
550065,6036,P00375436,F,26-35,15,B,1,20,NaN,NaN,137
550066,6038,P00375436,F,55+,1,C,0,20,NaN,NaN,365


In [34]:
df2[["prod_cat1","prod_cat2","prod_cat3"]] = df2[["prod_cat1","prod_cat2","prod_cat3"]].fillna(0)
df2

,user,prod,gender,age_group,job,city,marital,prod_cat1,prod_cat2,prod_cat3,purchase
0,1,P00069042,F,0-17,10,A,0,3,0.0,0.0,8370
1,1,P00248942,F,0-17,10,A,0,1,6.0,14.0,15200
2,1,P00087842,F,0-17,10,A,0,12,0.0,0.0,1422
3,1,P00085442,F,0-17,10,A,0,12,14.0,0.0,1057
4,2,P00285442,M,55+,16,C,0,8,0.0,0.0,7969
...,...,...,...,...,...,...,...,...,...,...,...
550063,6033,P00372445,M,51-55,13,B,1,20,0.0,0.0,368
550064,6035,P00375436,F,26-35,1,C,0,20,0.0,0.0,371
550065,6036,P00375436,F,26-35,15,B,1,20,0.0,0.0,137
550066,6038,P00375436,F,55+,1,C,0,20,0.0,0.0,365


In [40]:
df2.dtypes

user           int64
prod          object
gender        object
age_group     object
job            int64
city          object
marital        int64
prod_cat1      int64
prod_cat2    float64
prod_cat3    float64
purchase       int64
prod_cat      object
dtype: object

In [101]:
df2["prod_cat"] = df2["prod_cat1"].astype("str") + "-" + df2["prod_cat2"].astype("int").astype("str") + "-" + df2["prod_cat3"].astype("int").astype("str")

In [103]:
df2["prod_cat"].nunique()

237

In [72]:
df_21 = df2.loc[df2["user"]==1,:]
df_21.groupby("prod_cat")["prod_cat"].count()

prod_cat
1-2-5      1
1-2-9      1
1-6-14     1
1-8-17     1
12-0-0     2
12-14-0    1
14-0-0     1
16-0-0     1
2-4-8      1
20-0-0     1
3-0-0      1
3-4-0      3
3-4-12     5
3-4-5      2
4-8-0      1
4-8-9      1
5-0-0      1
5-15-0     1
6-8-16     1
8-0-0      7
8-17-0     1
Name: prod_cat, dtype: int64

In [68]:
df2

,user,prod,gender,age_group,job,city,marital,prod_cat1,prod_cat2,prod_cat3,purchase,prod_cat
0,1,P00069042,F,0-17,10,A,0,3,0.0,0.0,8370,3-0-0
1,1,P00248942,F,0-17,10,A,0,1,6.0,14.0,15200,1-6-14
2,1,P00087842,F,0-17,10,A,0,12,0.0,0.0,1422,12-0-0
3,1,P00085442,F,0-17,10,A,0,12,14.0,0.0,1057,12-14-0
4,2,P00285442,M,55+,16,C,0,8,0.0,0.0,7969,8-0-0
...,...,...,...,...,...,...,...,...,...,...,...,...
550063,6033,P00372445,M,51-55,13,B,1,20,0.0,0.0,368,20-0-0
550064,6035,P00375436,F,26-35,1,C,0,20,0.0,0.0,371,20-0-0
550065,6036,P00375436,F,26-35,15,B,1,20,0.0,0.0,137,20-0-0
550066,6038,P00375436,F,55+,1,C,0,20,0.0,0.0,365,20-0-0


In [112]:
df22 = df2.loc[df2["age_group"] == "26-35", :]


In [113]:
df222 = df22.groupby(["user","marital"])["prod_cat"].nunique().reset_index()
df222

,user,marital,prod_cat
0,3,0,18
1,5,1,43
2,8,1,32
3,9,0,31
4,11,0,34
...,...,...,...
2048,6030,1,33
2049,6034,0,8
2050,6035,0,61
2051,6036,1,123


In [114]:
stat_m0 = df222.loc[df222["marital"]==0, "prod_cat"].mean()
stat_m1 = df222.loc[df222["marital"]==1, "prod_cat"].mean()
round(abs(stat_m0 - stat_m1), 2)

0.13

# Q3

##  drop_duplicates()   
## 무조건 groupby는 아니다 ㅡ.ㅡ;

In [118]:
df3 = df.copy()

In [92]:
df3.columns

Index(['user', 'prod', 'gender', 'age_group', 'job', 'city', 'marital',
       'prod_cat1', 'prod_cat2', 'prod_cat3', 'purchase'],
      dtype='object')

In [97]:
df33 = df3.groupby("user")["gender","age_group","job","city","marital"]
pd.DataFrame(df33)

C:\Users\younl\AppData\Local\Temp/ipykernel_14316/2418306762.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df33 = df3.groupby("user")["gender","age_group","job","city","marital"]


,0,1
0,1,user prod gender age_group job ...
1,2,user prod gender age_group job ...
2,3,user prod gender age_group job ...
3,4,user prod gender age_group job ...
4,5,user prod gender age_group job ...
...,...,...
5886,6036,user prod gender age_group job ...
5887,6037,user prod gender age_group job ...
5888,6038,user prod gender age_group job ...
5889,6039,user prod gender age_group job ...


In [120]:
indep_l = ["user","gender","age_group","job","city","marital"]

In [121]:
df_q3_user = df3[indep_l]
df_q3_user = df_q3_user.drop_duplicates().reset_index(drop=True)
df_q3_user

,user,gender,age_group,job,city,marital
0,1,F,0-17,10,A,0
1,2,M,55+,16,C,0
2,3,M,26-35,15,A,0
3,4,M,46-50,7,B,1
4,5,M,26-35,20,A,1
...,...,...,...,...,...,...
5886,4588,F,26-35,4,C,0
5887,4871,M,18-25,12,C,0
5888,4113,M,36-45,17,C,0
5889,5391,M,26-35,7,A,0


In [123]:
df3["user"].nunique()

5891

In [124]:
df3_agg1 = df3.groupby("user")["prod"].nunique().reset_index()
df3_agg2 = df3.groupby("user")["purchase"].sum().reset_index()

In [127]:
df33 = df_q3_user.merge(df3_agg1,on="user")
df33 = df33.merge(df3_agg2,on="user")
df33

,user,gender,age_group,job,city,marital,prod,purchase
0,1,F,0-17,10,A,0,35,334093
1,2,M,55+,16,C,0,77,810472
2,3,M,26-35,15,A,0,29,341635
3,4,M,46-50,7,B,1,14,206468
4,5,M,26-35,20,A,1,106,821001
...,...,...,...,...,...,...,...,...
5886,4588,F,26-35,4,C,0,20,140990
5887,4871,M,18-25,12,C,0,12,108545
5888,4113,M,36-45,17,C,0,20,213550
5889,5391,M,26-35,7,A,0,7,60182


In [128]:
df33["gender"] = (df33["gender"]=="M")+0

In [131]:
arr_u = df33["age_group"].unique()
arr_u.sort()

In [132]:
ser_repl = pd.Series(range(len(arr_u)), index = arr_u)
ser_repl

0-17     0
18-25    1
26-35    2
36-45    3
46-50    4
51-55    5
55+      6
dtype: int64

In [140]:
pd.crosstab(df33["age_group"],
           df33["age_group"].replace(ser_repl))

age_group,0,1,2,3,4,5,6
age_group,,,,,,,
0,218,0,0,0,0,0,0
1,0,1069,0,0,0,0,0
2,0,0,2053,0,0,0,0
3,0,0,0,1167,0,0,0
4,0,0,0,0,531,0,0
5,0,0,0,0,0,481,0
6,0,0,0,0,0,0,372


In [135]:
df33["age_group"] = df33["age_group"].replace(ser_repl)

In [141]:
df_dum = pd.get_dummies(df33.drop(columns = "user"),
                        columns = ["job","city"], dtype="int")

In [143]:
arr_dum_nor = MinMaxScaler().fit_transform(df_dum)
arr_dum_nor[:1,]

array([[0.        , 0.        , 0.        , 0.02843137, 0.02739807,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        ]])

In [145]:
model_km = KMeans(n_clusters = 7, random_state = 123)
model_km.fit(arr_dum_nor)

KMeans(n_clusters=7, random_state=123)

In [146]:
round(silhouette_score(arr_dum_nor, model_km.labels_), 2)

0.18

In [147]:
pd.Series(model_km.labels_).value_counts()

1    1291
0    1004
5     980
2     868
6     703
3     652
4     393
dtype: int64